In [ ]:
import numpy as np
import tensorflow as tf
import math


In [ ]:
def process_data(image, label):
    return tf.cast(image, tf.float32)/255, tf.one_hot(label, 2, name="label", axis=-1)


def generate_data_tensor(image_train, label_train, train=True):

    def generator():
        for img, lbl in zip(image_train, label_train):
            yield img, lbl

    data = tf.data.Dataset.from_generator(generator,
                                          output_signature=(
                                              tf.TensorSpec(
                                                  shape=(512, 512, 3), dtype=tf.float32),
                                              tf.TensorSpec(shape=(512, 512), dtype=tf.int32)))
    data = data.map(
        process_data, num_parallel_calls=tf.data.AUTOTUNE)
    if train:
        data = data.shuffle(500)
    data = data.cache()
    data = data.batch(64)
    data = data.prefetch(tf.data.AUTOTUNE)
    return data


In [ ]:
image_train=np.load('../slides/image_train_augmented.npy')
label_train=np.load('../annotations/label_train_augmented.npy')
image_validation=np.load('../slides/image_validation.npy')
label_validation=np.load('../annotations/label_validation.npy')
image_test=np.load('../slides/image_test.npy')
label_test=np.load('../annotations/label_test.npy')

validation_data = generate_data_tensor(image_validation, label_validation)
test_data = generate_data_tensor(image_test, image_test, train=False)


steps_per_epoch = math.ceil(len(image_train) / 64)


# Definisci la funzione per addestrare e valutare un modello con un dato tasso di apprendimento
def train_and_evaluate_segnet(image_train, label_train, validation_data, learning_rate, steps_per_epoch):
    # Costruisci il modello
    model = SegNet()
    # Compila il modello con la funzione di perdita e l'ottimizzatore appropriati
    model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate), metrics=["accuracy", "Precision", "Recall", "FalseNegatives",
                                                                                                               "FalsePositives", "TrueNegatives", "TruePositives"])
    print("testing a model")
    # Addestra il modello sul training set
    image_train_size = len(image_train) // 20
    label_train_size = len(label_train) // 20
    
    for i in range(20):
        train_data = [];
        if i == 19:
            train_data=generate_data_tensor(image_train=image_train[i*image_train_size:], label_train=label_train[i*label_train_size:])
        else:
            train_data=generate_data_tensor(image_train=image_train[i*image_train_size: (i+1)*image_train_size], label_train=label_train[i*label_train_size:(i+1)*label_train_size])
        
        model.fit(train_data, epochs=50, steps_per_epoch=steps_per_epoch)
        
  
    # Valuta il modello sul validation set
    evals = model.evaluate(validation_data)
    return model, evals[1]


def train_and_evaluate_unet(image_train, label_train, validation_data, learning_rate, steps_per_epoch):
    # Costruisci il modello
    model = Unet()
    # Compila il modello con la funzione di perdita e l'ottimizzatore appropriati
    model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate), metrics=["accuracy", "Precision", "Recall", "FalseNegatives",
                                                                                                               "FalsePositives", "TrueNegatives", "TruePositives"])
    print("testing a model")
    # Addestra il modello sul training set
    image_train_size = len(image_train) // 20
    label_train_size = len(label_train) // 20
    
    for i in range(20):
        train_data = [];
        if i == 19:
            train_data=generate_data_tensor(image_train=image_train[i*image_train_size:], label_train=label_train[i*label_train_size:])
        else:
            train_data=generate_data_tensor(image_train=image_train[i*image_train_size: (i+1)*image_train_size], label_train=label_train[i*label_train_size:(i+1)*label_train_size])
        
        model.fit(train_data, epochs=50, steps_per_epoch=steps_per_epoch)
    # Valuta il modello sul validation set
    evals = model.evaluate(validation_data)
    return model, evals[1]


best_accuracy = 0.0
best_learning_rate = None
best_models = None
best_history = None

learning_rates = [0.001, 0.01, 0.1]


# Valuta ogni tasso di apprendimento e seleziona il migliore
for learning_rate in learning_rates:
    print("creating a model")
    model, accuracy = train_and_evaluate_segnet(
        image_train, label_train, validation_data, learning_rate, steps_per_epoch)
    print(f"Learning Rate: {learning_rate}, Accuracy: {accuracy}")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_learning_rate = learning_rate
        best_model = model

best_models.save('../saved_model/segnet')

